In [ ]:
from scipy import io

classes = io.loadmat('/home/ashok/Downloads/car_devkit/devkit/cars_meta.mat')
len(classes['class_names'][0])

In [ ]:
annotations = io.loadmat('/home/ashok/Downloads/car_devkit/devkit/cars_train_annos.mat')
print(len(annotations['annotations'][0]))

In [ ]:
from tensorflow import keras
import tensorflow as tf
import pandas as pd
import numpy as np

In [ ]:
# tf.config.gpu_options.allow_growth = True
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [ ]:
class_list = [data[0] for data in classes['class_names'][0]]
print(len(class_list))
class_list

In [ ]:
f_names = [data[5] for data in annotations['annotations'][0]]
f_names = np.squeeze(f_names)

In [ ]:
c_ints = [data[4] for data in annotations['annotations'][0]]
c_ints = np.squeeze(c_ints)

## Create a List of Classes corresponding to c_ints

In [ ]:
class_names = [class_list[index-1] for index in c_ints]
print(len(class_names))

In [ ]:
df = pd.DataFrame([f_names, class_names]).T
df.columns = ['filename', 'class']
df[:20]

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

In [ ]:
datagen = ImageDataGenerator(
    rotation_range=20,
    zoom_range = 0.1,  
    width_shift_range=0.1, 
    height_shift_range=0.1,
    rescale=1./255,
    validation_split=0.1)

train_generator = datagen.flow_from_dataframe(
    dataframe = df, 
    directory = "/home/ashok/Downloads/cars_train", 
    x_col = "filename", 
    y_col = "class", 
    class_mode = "categorical", 
    target_size = (256,256), 
    subset='training',
    batch_size = 32)

valid_generator = datagen.flow_from_dataframe(
    dataframe = df, 
    directory = "/home/ashok/Downloads/cars_train", 
    x_col = "filename", 
    y_col = "class", 
    class_mode = "categorical", 
    target_size = (256,256), 
    subset='validation',
    batch_size = 32)

In [ ]:
model = keras.Sequential([
    keras.layers.Conv2D(input_shape =(256,256,3), filters=16, kernel_size=(3,3), activation = 'relu',
                        strides=(1,1), data_format = "channels_last",),
    
    keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation = 'relu',
                        strides=(1,1), data_format = "channels_last"),
    keras.layers.Dropout(0.2),
    keras.layers.BatchNormalization(),
    
    keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation = 'relu',
                        strides=(1,1), data_format = "channels_last"),
    keras.layers.Dropout(0.2),
    keras.layers.BatchNormalization(),
    
    keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation = 'relu',
                        strides=(1,1), data_format = "channels_last"),
    keras.layers.MaxPool2D(2,2),
    keras.layers.BatchNormalization(),
    
    keras.layers.Conv2D(filters=64, kernel_size=(3,3), activation = 'relu',
                        strides=(1,1), data_format = "channels_last"),
    keras.layers.Dropout(0.4),
    keras.layers.BatchNormalization(),
    
    keras.layers.Conv2D(filters=64, kernel_size=(3,3), activation = 'relu',
                        strides=(1,1), data_format = "channels_last"),
    keras.layers.Dropout(0.4),
    keras.layers.BatchNormalization(),
    
    keras.layers.Conv2D(filters=64, kernel_size=(3,3), activation = 'relu',
                        strides=(1,1), data_format = "channels_last"),
    keras.layers.MaxPool2D(2,2),
    keras.layers.Dropout(0.4),
    
    keras.layers.Conv2D(filters=128, kernel_size=(3,3), activation = 'relu',
                        strides=(1,1), data_format = "channels_last"),
    keras.layers.Dropout(0.4),
    keras.layers.BatchNormalization(),
    
    keras.layers.Conv2D(filters=128, kernel_size=(3,3), activation = 'relu',
                        strides=(1,1), data_format = "channels_last"),
    keras.layers.Dropout(0.5),
    keras.layers.BatchNormalization(),
    
    keras.layers.Conv2D(filters=128, kernel_size=(3,3), activation = 'relu',
                        strides=(1,1), data_format = "channels_last"),
    keras.layers.Dropout(0.5),
    keras.layers.BatchNormalization(),
    
    keras.layers.Conv2D(filters=128, kernel_size=(3,3), activation = 'relu',
                        strides=(1,1), data_format = "channels_last"),
    keras.layers.Dropout(0.5),
    keras.layers.MaxPool2D(2,2),
    keras.layers.BatchNormalization(),
    
    keras.layers.Conv2D(filters=256, kernel_size=(3,3), activation = 'relu',
                        strides=(1,1), data_format = "channels_last"),
    keras.layers.Dropout(0.5),
    keras.layers.BatchNormalization(),
    
    keras.layers.Conv2D(filters=256, kernel_size=(3,3), activation = 'relu',
                        strides=(1,1), data_format = "channels_last"),
    keras.layers.Dropout(0.5),
    keras.layers.MaxPool2D(2,2),
    keras.layers.BatchNormalization(),
    
    keras.layers.Flatten(),
    
    keras.layers.Dense(256, activation = 'relu'),
    keras.layers.Dense(196, activation="softmax")
])

In [ ]:
model.summary()

In [ ]:
model.compile(Adam(lr=0.0001),
              loss="categorical_crossentropy",
              metrics=["accuracy"])

In [ ]:
STEP_SIZE_TRAIN = train_generator.n//train_generator.batch_size
STEP_SIZE_VALID = valid_generator.n//valid_generator.batch_size
history = model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=10)

In [ ]:
model.save("car_classification.h5")

In [ ]:
from tensorflow.keras.models import load_model
model = load_model('car_classification.h5')

In [ ]:
import cv2
img = cv2.imread('/home/ashok/Downloads/cars_train/00019.jpg')
image_x = 256
image_y = 256
img = cv2.resize(img, (image_x, image_y))
img = np.array(img, dtype=np.float32)
img = np.reshape(img, (-1, image_x, image_y, 3))

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
image = mpimg.imread("/home/ashok/Downloads/cars_test/00005.jpg")
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
plt.show()

In [ ]:
pred = model.predict(img)
class_list[np.argmax(pred)]
